In [1]:
2+2

4

In [2]:
from pyflink.table import (
    DataTypes, TableEnvironment, EnvironmentSettings, 
    CsvTableSource, CsvTableSink, WriteMode
)

In [3]:
env_settings = (
    EnvironmentSettings.new_instance()
                       .in_batch_mode()
                       .use_blink_planner()
                       .build()
)

In [4]:
env_settings

In [5]:
tbl_env = TableEnvironment.create(env_settings)

In [6]:
in_field_names = ['seller_id', 'product', 'quantity', 'product_price', 'sales_date']

In [7]:
in_field_types = [DataTypes.STRING(), DataTypes.STRING(), DataTypes.INT(), DataTypes.DOUBLE(), DataTypes.DATE()]

In [8]:
source = CsvTableSource(
        './input',
        in_field_names,
        in_field_types,
        ignore_first_line=True
    )

In [9]:
source

In [10]:
tbl_env.register_table_source('locale_product_sales', source)

In [11]:
out_field_names = ['seller_id', 'revenue']

In [12]:
out_field_types = [DataTypes.STRING(), DataTypes.DOUBLE()]

In [13]:
sink = CsvTableSink(
        out_field_names,
        out_field_types,
        './table-api-output/revenue.csv',
        num_files=1,
        write_mode=WriteMode.OVERWRITE
    )

In [14]:
tbl_env.register_table_sink('locale_revenue', sink)

In [15]:
tbl = tbl_env.from_path('locale_product_sales')

In [16]:
sales_tbl = tbl.add_columns(
        (tbl.quantity * tbl.product_price).alias('sales')
    )

In [17]:
sales_tbl.print_schema()

(
  `seller_id` STRING,
  `product` STRING,
  `quantity` INT,
  `product_price` DOUBLE,
  `sales_date` DATE,
  `sales` DOUBLE
)


In [18]:
output_tbl = (
    sales_tbl.group_by(sales_tbl.seller_id)
             .select(sales_tbl.seller_id,sales_tbl.sales.sum.alias('revenue'))
)

In [19]:
output_tbl.print_schema()

(
  `seller_id` STRING,
  `revenue` DOUBLE
)


In [20]:
output_tbl.execute_insert('locale_revenue').wait()

In [21]:
import pandas as pd

In [22]:
df = pd.read_csv('./table-api-output/revenue.csv')

In [23]:
df

,OMA,650.1400000000001
0,LNK,502.57
